# Sprint2: baseline calibration eval (ECE/Brier + reliability plots)

In [ ]:
# Librerías
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve
import numpy as np
from sklearn.calibration import CalibratedClassifierCV
import json


# 1) Obtención y Calidad de la información

## Load data

In [ ]:
def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "pyproject.toml").exists():
            return p
    raise FileNotFoundError("Repo root not found (pyproject.toml missing).")

ROOT = find_repo_root(Path.cwd())
DATA_PATH = ROOT / "data" / "processed" / "train.parquet"

print("Repo root:", ROOT)
print("Data path:", DATA_PATH)
print("Exists:", DATA_PATH.exists())

df = pd.read_parquet(DATA_PATH)
df.shape


In [ ]:
# Keep issue_d for temporal validation (OOT)
if "issue_d" not in df.columns:
    raise KeyError("issue_d not found in raw dataset. Needed for OOT validation.")

df["issue_d"] = pd.to_datetime(df["issue_d"], format="%b-%Y", errors="coerce")

parse_rate = df["issue_d"].notna().mean()
if parse_rate < 0.95:
    raise ValueError(f"issue_d parse_rate too low: {parse_rate:.3f}. Check raw format.")

# Optional: month bucket for easy plots (YYYY-MM)
df["issue_month"] = df["issue_d"].dt.to_period("M").astype(str)


## Check de variable de tiempo

In [ ]:
candidates = [
    c for c in df.columns
    if any(k in c.lower() for k in ["issue", "origination", "date", "time", "month", "year"])
]
candidates

In [ ]:
df.columns.tolist()

In [ ]:
df.head()

In [ ]:
# Rango de tiempo de la información
df["issue_d"] = pd.to_datetime(df["issue_d"], format="%b-%Y", errors="coerce")
df["issue_d"].min(), df["issue_d"].max()


## Observaciones por mes

In [ ]:
# --- Ensure issue_d is datetime (if it's already datetime64[ns], this is safe) ---
df["issue_d"] = pd.to_datetime(df["issue_d"], errors="coerce")
df = df.dropna(subset=["issue_d"])

# --- Create month key (cohort/vintage) ---
df["issue_month"] = df["issue_d"].dt.to_period("M").dt.to_timestamp()

# --- Cohort size per month ---
cohort_size = (
    df.groupby("issue_month")
      .size()
      .rename("n_loans")
      .reset_index()
      .sort_values("issue_month")
)

cohort_size.head(), cohort_size.tail()


In [ ]:
# Plot cohort size
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(cohort_size["issue_month"], cohort_size["n_loans"])
ax.set_title("Cohort size by issue month")
ax.set_xlabel("Issue month")
ax.set_ylabel("Number of loans")
plt.show()


## Default por mes

In [ ]:
# Default rate por mes
cohort_default = (
    df.groupby("issue_month")["target"]
      .agg(default_rate="mean", n_loans="size")
      .reset_index()
      .sort_values("issue_month")
)

cohort_default.head(), cohort_default.tail()

In [ ]:
# Plot default rate
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(cohort_default["issue_month"], cohort_default["default_rate"])
ax.set_title("Default rate by issue month")
ax.set_xlabel("Issue month")
ax.set_ylabel("Default rate")
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10, 7), sharex=True)

# Top: default rate
ax1.plot(cohort_default["issue_month"], cohort_default["default_rate"])
ax1.set_title("Default rate & cohort size by issue month")
ax1.set_ylabel("Default rate")

# Bottom: cohort size
ax2.plot(cohort_size["issue_month"], cohort_size["n_loans"])
ax2.set_xlabel("Issue month")
ax2.set_ylabel("Number of loans (log)")

# Opcional: ayuda mucho cuando hay meses con tamaños muy distintos
ax2.set_yscale("log")  # comenta esta línea si prefieres escala lineal

plt.show()


In [ ]:
# Suavido de la tasa (rolling “pooling” ponderado)
tmp = cohort_default.copy()

# Rolling 6-month pooled default rate (weighted by n_loans)
window = 6
tmp["default_rate_roll6"] = (
    (tmp["default_rate"] * tmp["n_loans"]).rolling(window, min_periods=3).sum()
    / tmp["n_loans"].rolling(window, min_periods=3).sum()
)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(tmp["issue_month"], tmp["default_rate"], alpha=0.4)
ax.plot(tmp["issue_month"], tmp["default_rate_roll6"])
ax.set_title("Default rate by issue month (raw vs 6m pooled)")
ax.set_xlabel("Issue month")
ax.set_ylabel("Default rate")
plt.show()


# 2) Temporal split (Train / Val / Test)

## Tamaño de datasets

In [ ]:
# --- Monthly cohort table (para elegir cortes por volumen, no por fechas arbitrarias) ---
cohort = (
    df.groupby("issue_month")["target"]
      .agg(n_loans="size", default_rate="mean")
      .reset_index()
      .sort_values("issue_month")
)
cohort["cum_loans"] = cohort["n_loans"].cumsum()
total = cohort["cum_loans"].iloc[-1]

# 70% train, 15% val, 15% test (por cantidad de loans)
train_end = cohort.loc[cohort["cum_loans"] <= total * 0.70, "issue_month"].max()
val_end   = cohort.loc[cohort["cum_loans"] <= total * 0.85, "issue_month"].max()

print("Cutoffs:")
print("  train_end =", train_end)
print("  val_end   =", val_end)

In [ ]:
# Create datasets
train_df = df[df["issue_month"] <= train_end].copy()
val_df   = df[(df["issue_month"] > train_end) & (df["issue_month"] <= val_end)].copy()
test_df  = df[df["issue_month"] > val_end].copy()

def _summ(name, d):
    return {
        "split": name,
        "rows": len(d),
        "min_month": d["issue_month"].min(),
        "max_month": d["issue_month"].max(),
        "default_rate": float(d["target"].mean()),
    }

pd.DataFrame([_summ("train", train_df), _summ("val", val_df), _summ("test", test_df)])

# 3) Entrenar y evaluar baseline

## Entrenamiento del modelo

In [ ]:
# --- Features/target (IMPORTANTE: quitamos issue_d/issue_month) ---
DROP_COLS = ["target", "issue_d", "issue_month"]

X_train = train_df.drop(columns=DROP_COLS)
y_train = train_df["target"].astype(int)

X_val = val_df.drop(columns=DROP_COLS)
y_val = val_df["target"].astype(int)

X_test = test_df.drop(columns=DROP_COLS)
y_test = test_df["target"].astype(int)

cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = [c for c in X_train.columns if c not in cat_cols]

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop",
)

# --- Baseline LGBM (simple pero sólido) ---
model = lgb.LGBMClassifier(
    n_estimators=600,
    learning_rate=0.05,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)

clf = Pipeline(steps=[
    ("prep", preprocess),
    ("model", model),
])

clf.fit(X_train, y_train)

def eval_split(name, X, y):
    p = clf.predict_proba(X)[:, 1]
    return {
        "split": name,
        "auc": roc_auc_score(y, p),
        "brier": brier_score_loss(y, p),
        "n": len(y),
        "default_rate": float(y.mean()),
    }, p

rows = []
_, p_train = eval_split("train", X_train, y_train)
r_val, p_val = eval_split("val", X_val, y_val)
r_test, p_test = eval_split("test", X_test, y_test)

rows.append(r_val)
rows.append(r_test)
pd.DataFrame(rows)


## Reliability diagram (calibración) en TEST

In [ ]:
prob_true, prob_pred = calibration_curve(y_test, p_test, n_bins=10, strategy="quantile")

plt.figure()
plt.plot(prob_pred, prob_true, marker="o")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.title("Calibration (reliability diagram) - TEST (OOT)")
plt.xlabel("Mean predicted probability")
plt.ylabel("Empirical default rate")
plt.show()


# 4) Calibración

## Métricas base y función auxiliar

In [ ]:
def eval_probs(name, y, p):
    return {
        "model": name,
        "auc": roc_auc_score(y, p),
        "brier": brier_score_loss(y, p),
        "n": len(y),
        "default_rate": float(np.mean(y)),
        "avg_pred": float(np.mean(p)),
    }

def expected_calibration_error(y, p, n_bins=10):
    # ECE estándar: binning uniforme en [0,1]
    y = np.asarray(y)
    p = np.asarray(p)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    idx = np.digitize(p, bins) - 1
    ece = 0.0
    for b in range(n_bins):
        mask = idx == b
        if mask.sum() == 0:
            continue
        acc = y[mask].mean()
        conf = p[mask].mean()
        w = mask.mean()
        ece += w * abs(acc - conf)
    return float(ece)


## Baseline probs (sin calibrar)

In [ ]:
p_val_base = clf.predict_proba(X_val)[:, 1]
p_test_base = clf.predict_proba(X_test)[:, 1]

rows = []
rows.append(eval_probs("baseline", y_val, p_val_base) | {"split": "val", "ece": expected_calibration_error(y_val, p_val_base)})
rows.append(eval_probs("baseline", y_test, p_test_base) | {"split": "test", "ece": expected_calibration_error(y_test, p_test_base)})

pd.DataFrame(rows)[["split","model","auc","brier","ece","default_rate","avg_pred","n"]]


## Platt & Isotonic (calibrar en VAL, evaluar en TEST OOT)

In [ ]:
# Platt scaling (sigmoid)
cal_sigmoid = CalibratedClassifierCV(estimator=clf, method="sigmoid", cv="prefit")
cal_sigmoid.fit(X_val, y_val)

p_val_sig = cal_sigmoid.predict_proba(X_val)[:, 1]
p_test_sig = cal_sigmoid.predict_proba(X_test)[:, 1]

# Isotonic
cal_iso = CalibratedClassifierCV(estimator=clf, method="isotonic", cv="prefit")
cal_iso.fit(X_val, y_val)

p_val_iso = cal_iso.predict_proba(X_val)[:, 1]
p_test_iso = cal_iso.predict_proba(X_test)[:, 1]

rows = []
for split_name, y, p_base, p_sig, p_iso in [
    ("val", y_val, p_val_base, p_val_sig, p_val_iso),
    ("test", y_test, p_test_base, p_test_sig, p_test_iso),
]:
    rows.append(eval_probs("baseline", y, p_base) | {"split": split_name, "ece": expected_calibration_error(y, p_base)})
    rows.append(eval_probs("platt_sigmoid", y, p_sig) | {"split": split_name, "ece": expected_calibration_error(y, p_sig)})
    rows.append(eval_probs("isotonic", y, p_iso) | {"split": split_name, "ece": expected_calibration_error(y, p_iso)})

df_metrics = pd.DataFrame(rows)[["split","model","auc","brier","ece","default_rate","avg_pred","n"]]
df_metrics.sort_values(["split","model"])


## Curvas de calibración (antes vs después) en TEST OOT

In [ ]:
# ECE
def expected_calibration_error(y_true, p_pred, n_bins=10, strategy="quantile"):
    """
    ECE (Expected Calibration Error):
    sum_b (|b|/N) * |acc(b) - conf(b)|
    where acc(b)=mean(y), conf(b)=mean(p) within bin.
    """
    y_true = np.asarray(y_true)
    p_pred = np.asarray(p_pred)

    df_ = pd.DataFrame({"y": y_true, "p": p_pred})

    if strategy == "quantile":
        # bins con igual cantidad de observaciones (maneja duplicados con drop)
        df_["bin"] = pd.qcut(df_["p"], q=n_bins, duplicates="drop")
    elif strategy == "uniform":
        df_["bin"] = pd.cut(df_["p"], bins=n_bins, include_lowest=True)
    else:
        raise ValueError("strategy must be 'quantile' or 'uniform'")

    g = df_.groupby("bin", observed=False)
    bin_stats = g.agg(
        n=("p", "size"),
        conf=("p", "mean"),
        acc=("y", "mean"),
    ).reset_index()

    bin_stats["w"] = bin_stats["n"] / len(df_)
    bin_stats["abs_gap"] = (bin_stats["acc"] - bin_stats["conf"]).abs()

    ece = float((bin_stats["w"] * bin_stats["abs_gap"]).sum())
    return ece, bin_stats


In [ ]:
# Cálculo de la ECE
ece_base, bins_base = expected_calibration_error(y_test, p_test_base, n_bins=10, strategy="quantile")
ece_platt, bins_platt = expected_calibration_error(y_test, p_test_sig, n_bins=10, strategy="quantile")
ece_iso, bins_iso = expected_calibration_error(y_test, p_test_iso, n_bins=10, strategy="quantile")

pd.DataFrame({
    "model": ["baseline", "platt_sigmoid", "isotonic"],
    "ece_quantile_10bins": [ece_base, ece_platt, ece_iso],
})


In [ ]:
def plot_calibration(y, probs_dict, title, n_bins=10):
    plt.figure()
    plt.plot([0, 1], [0, 1], "--")  # perfect calibration line

    for name, p in probs_dict.items():
        frac_pos, mean_pred = calibration_curve(y, p, n_bins=n_bins, strategy="quantile")
        plt.plot(mean_pred, frac_pos, marker="o", label=name)

    plt.title(title)
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Empirical default rate")
    plt.legend()
    plt.show()

plot_calibration(
    y_test,
    {
        "baseline": p_test_base,
        "platt_sigmoid": p_test_sig,
        "isotonic": p_test_iso,
    },
    title="Calibration (reliability diagram) - TEST (OOT)"
)


# 5) Cierre del baseline

## Guardar métricas baseline y calibrados en un artifact

In [ ]:
out = Path("artifacts/reports")
out.mkdir(parents=True, exist_ok=True)

# df_metrics ya lo tienes (split/model/auc/brier/ece/...)
df_metrics.to_csv(out / "s2_baseline_calibration_metrics.csv", index=False)

with open(out / "s2_baseline_notes.json", "w") as f:
    json.dump({
        "note": "Baseline + calibration on VAL; evaluated on TEST (OOT).",
        "calibration_bins": 10,
        "calibration_strategy": "quantile",
        "models": ["baseline", "platt_sigmoid", "isotonic"],
    }, f, indent=2)


## Anity check de “avg_pred vs default_rate” (por split)

In [ ]:
df_metrics.assign(gap=lambda d: d["avg_pred"] - d["default_rate"])[
    ["split","model","default_rate","avg_pred","gap","auc","brier","ece","n"]
].sort_values(["split","model"])
